## ライブラリインポート

In [1]:
import numpy as np
import keras
from keras.models import  Model,load_model
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Input,GlobalAveragePooling2D,BatchNormalization
from keras.optimizers import Adam
# from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import pickle

Using TensorFlow backend.
c:\users\12810649\appdata\local\continuum\anaconda3\envs\tf1.1.4\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\12810649\appdata\local\continuum\anaconda3\envs\tf1.1.4\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\12810649\appdata\local\continuum\anaconda3\envs\tf1.1.4\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint1

## データ整形

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# データ次元確認
print('x_train.shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

x_train.shape: (50000, 32, 32, 3)
x_test.shape: (10000, 32, 32, 3)
y_train.shape: (50000, 1)
y_test.shape: (10000, 1)


In [4]:
# 正規化
x_train = x_train/255.
x_test = x_test/255.

# one hot ベクトル化
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## モデル作成

In [5]:
def create_bench_model():
    inputs = Input(shape = (32,32,3))
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(inputs)
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
    x = Dropout(0.25)(x)
    x = MaxPooling2D()(x)

    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = Dropout(0.25)(x)
    x = MaxPooling2D()(x)

    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(1024,activation = "relu")(x)
    x = Dropout(0.25)(x)
    y = Dense(10,activation = "softmax")(x)
    
    model = Model(input = inputs, output = y)

    return model

In [6]:
model = create_bench_model()
model.compile(loss = "categorical_crossentropy",optimizer = Adam(), metrics = ["accuracy"])
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       73856     
____________________

c:\users\12810649\appdata\local\continuum\anaconda3\envs\tf1.1.4\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [7]:
history = model.fit(x_train,
                    y_train,
                    batch_size=32,
                    epochs=20,
                    validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 14s 338us/step - loss: 2.0683 - acc: 0.1998 - val_loss: 1.6880 - val_acc: 0.3683
Epoch 2/20
40000/40000 [==============================] - 10s 260us/step - loss: 1.4304 - acc: 0.4707 - val_loss: 1.3245 - val_acc: 0.5293
Epoch 3/20
40000/40000 [==============================] - 10s 257us/step - loss: 1.1237 - acc: 0.5989 - val_loss: 1.1076 - val_acc: 0.6141
Epoch 4/20
40000/40000 [==============================] - 10s 255us/step - loss: 0.9419 - acc: 0.6686 - val_loss: 0.9808 - val_acc: 0.6641
Epoch 5/20
40000/40000 [==============================] - 10s 258us/step - loss: 0.8121 - acc: 0.7135 - val_loss: 0.8049 - val_acc: 0.7252
Epoch 6/20
40000/40000 [==============================] - 10s 251us/step - loss: 0.7228 - acc: 0.7446 - val_loss: 0.8173 - val_acc: 0.7098
Epoch 7/20
40000/40000

In [8]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 73us/step
Test loss: 0.7853299885749817
Test accuracy: 0.7771


In [16]:
import os
# Save model and weights
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'cifar10_tf114_functional_api_ver.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\12810649\Projects\dnn_samples\cifar10\models\cifar10_tf114_functional_api_ver.h5 
